In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## Padding a 3D mesh

This notebook shows how to pad a 3D mesh.

In [ ]:
import numpy as np
from nanomesh import Volume

n = 24
m = 12

data = np.ones([2*n, 5*n, 3*n], dtype=int)
data[m:3*m, m:3*m, 0:3*n] = 0

print(data.shape)

vol = Volume(data)
vol.show_slice()

# generate contours
from nanomesh.mesh3d import Mesher3D

mesher = Mesher3D(vol)
mesher.generate_contour()

contour = mesher.contour

### Padding a mesh

First step is to create a bounding box for the padded area (below).

In [ ]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [ ]:
import copy
from nanomesh.mesh_utils_3d import pad3d
from nanomesh.mesh3d import close_side, BoundingBox

pad_side = 'back'

contour2 = copy.copy(contour)

edge_col, edge_value, extra_coords = pad3d(contour2, side=pad_side, width=20, opts='-pAq1.2 -a100', label=3)

n_points = len(contour2.points)
points = np.vstack([contour2.points, extra_coords])

new_triangles = []

cols = {
    'top': (0,1,1,0),
    'bottom': (0,1,1,0),
    'left': (0,2,2,0),
    'right': (0,2,2,0),
    'front': (1,2,2,1),
    'back': (1,2,2,1),
}[pad_side]

for corner, col in zip(extra_coords, cols):
    connect_to = np.argwhere(
        (points[:, edge_col] == edge_value) & 
        (points[:,col] == corner[col])
    )
    
    additional_points = np.argwhere(extra_coords[:,col] == corner[col]) + n_points
   
    first, last = additional_points    
    first_point = points[first]    

    sorted_by_distance = np.argsort(np.linalg.norm(first_point - points[connect_to].squeeze(), axis=1))
    connect_to = connect_to[sorted_by_distance]
    connect_to = np.vstack([connect_to, last])

    for pair in pairwise(connect_to):
        tri = np.vstack((first, pair))
        new_triangles.append(tri)

new_triangles = np.array(new_triangles).squeeze()

cells = np.vstack([contour2.cells, new_triangles])

In [ ]:
from nanomesh.mesh_container import TriangleMesh
padded_mesh = TriangleMesh(points=points, cells=cells)

padded_mesh.plot_itk()

In [ ]:
np.arange(25).reshape(5,5).size

In [ ]:
from nanomesh.mesh_utils import simple_triangulate


simple_triangulate(points=points)


In [ ]:
np.argwhere(extra_coords[:,col] == corner[col]) + n_points

In [ ]:
(points[:, edge_col] == edge_value)

In [ ]:
(points[:,col] == corner[col])

In [ ]:



coords, is_edge = pad3d(padded_mesh, side=pad_side, width=20, opts='-pAq1.2 -a100', label=3)

# bounding box for padded area
bbox = BoundingBox.from_points(coords)

# get consistent set of points / cells for padded area
edge_indices = np.argwhere(is_edge).flatten()

has_point_on_edge = np.isin(tetras.cells, edge_indices)
has_face_on_edge = np.sum(has_point_on_edge, axis=1)==3

print(edge_indices.shape)
print(has_point_on_edge.shape)
print(has_face_on_edge.shape)

edge_surface_triangles = tetras.cells[has_face_on_edge][has_point_on_edge[has_face_on_edge]].reshape(-1,3)

print(edge_surface_triangles.shape)

# assert that all triangles have the same z value.
assert np.all(tetras.points[edge_surface_triangles][:,:,2].flatten() == 71)

unq_arr, unq_tags = np.unique(edge_surface_triangles, return_inverse=1)

relabeled = np.argsort(unq_arr)[unq_tags].reshape(-1,3)
relabeled

# Generate triangle mesh corresponding to specified side, 
# TODO: refactor this into function, i.e.
# def extract_side(mesh, side='top'):
#     pass
from nanomesh.mesh_container import TriangleMesh
side_mesh = TriangleMesh(points=coords, cells=relabeled)
# side_mesh.drop_third_dimension()
# side_mesh.plot()

sides = 'top', 'bottom', 'left', 'right', 'front', 'back'
for side in sides:
    if side == pad_side:
        continue
    
    side_mesh = close_side(side_mesh, side=side, bbox=bbox)

side_mesh.plot_itk()

### Tetrahedralize mesh

This seems to work. Ensure that points/tetrahedras have the same labeling as in the triangle mesh above.

In [ ]:
vol_mesh = side_mesh.tetrahedralize()
vol_mesh.plot_itk()

### Next step: merge meshes back

In [ ]:
# tetramesh
tetras = mesher.tetrahedralize(opts='-pAq1.2 -a100')
tetras.metadata['labels'] = tetras.metadata['tetgenRef']
tetras.plot_pyvista(jupyter_backend='static', show_edges=True)  # Use .plot_itk() for an interactive view 